<a href="https://colab.research.google.com/github/invegat/DS-Unit-2-Sprint-2-Linear-Regression/blob/master/module1-OLS-regression/Log_linear_Regression_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intermediate Linear Regression Practice

## Use a Linear Regression model to get the lowest RMSE possible on the following dataset:

[Dataset Folder](https://github.com/ryanleeallred/datasets/tree/master/Ames%20Housing%20Data)

[Raw CSV](https://raw.githubusercontent.com/ryanleeallred/datasets/master/Ames%20Housing%20Data/train.csv)

## You model must include (at least):
- A log-transformed y variable
- Two polynomial features
- One interaction feature
- 10 other engineered features

What is the lowest Root-Mean-Squared Error that you are able to obtain? Share your best RMSEs in Slack!

Notes:

There may be some data cleaning that you need to do on some features of this dataset. Linear Regression will only accept numeric values and will not accept

Note* There may not be a clear candidate for an interaction term in this dataset. Include one anyway, sometimes it's a good practice for predictive modeling feature engineering in general. 

In [0]:
!pip install -U matplotlib

Requirement already up-to-date: matplotlib in /usr/local/lib/python3.6/dist-packages (3.0.2)


In [0]:
%matplotlib inline
import matplotlib
matplotlib._version.sys.version

'3.6.7 (default, Oct 22 2018, 11:32:17) \n[GCC 8.2.0]'

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from datetime import datetime
import math

In [0]:
binaries = [f'{i:08b}' for i in range(1,100)]

In [0]:
pd.set_option('display.max_columns', 10000)
df = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/Ames%20Housing%20Data/train.csv')
print(df.shape)
df.head()

(1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [0]:
cats = df.select_dtypes(include=['category','object']).columns.unique().tolist()

In [0]:

# cats = ['MSZoning','Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
#         'Condition1', 'Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType',
#         'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond','BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
#         'Heating','HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish',
#         'GarageQual', 'GarageCond', 'PavedDrive','PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition' ]
for c in cats:
  cl = c[0].lower() + c[1:] + '-'
  types = df[c].unique().tolist()  
  for i in range(min(8,len(types)+1)):
     df[cl + str(i)] = df[c].apply(lambda s: int(('00000000' if pd.isnull(s) else binaries[types.index(s)])[i]))   
dfb = df.drop(cats,inplace=False, axis=1)
dfb.head()




,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice,mSZoning-0,mSZoning-1,mSZoning-2,mSZoning-3,mSZoning-4,mSZoning-5,street-0,street-1,street-2,alley-0,alley-1,alley-2,alley-3,lotShape-0,lotShape-1,lotShape-2,lotShape-3,lotShape-4,landContour-0,landContour-1,landContour-2,landContour-3,landContour-4,utilities-0,utilities-1,utilities-2,lotConfig-0,lotConfig-1,lotConfig-2,lotConfig-3,lotConfig-4,lotConfig-5,landSlope-0,landSlope-1,landSlope-2,landSlope-3,neighborhood-0,neighborhood-1,neighborhood-2,neighborhood-3,neighborhood-4,neighborhood-5,neighborhood-6,neighborhood-7,condition1-0,condition1-1,condition1-2,condition1-3,condition1-4,condition1-5,condition1-6,condition1-7,condition2-0,condition2-1,condition2-2,condition2-3,condition2-4,condition2-5,condition2-6,condition2-7,bldgType-0,bldgType-1,bldgType-2,bldgType-3,bldgType-4,bldgType-5,houseStyle-0,houseStyle-1,houseStyle-2,houseStyle-3,houseStyle-4,houseStyle-5,houseStyle-6,houseStyle-7,roofStyle-0,roofStyle-1,roofStyle-2,roofStyle-3,roofStyle-4,roofStyle-5,roofStyle-6,roofMatl-0,roofMatl-1,roofMatl-2,roofMatl-3,roofMatl-4,roofMatl-5,roofMatl-6,roofMatl-7,exterior1st-0,exterior1st-1,exterior1st-2,exterior1st-3,exterior1st-4,exterior1st-5,exterior1st-6,exterior1st-7,exterior2nd-0,exterior2nd-1,exterior2nd-2,exterior2nd-3,exterior2nd-4,exterior2nd-5,exterior2nd-6,exterior2nd-7,masVnrType-0,masVnrType-1,masVnrType-2,masVnrType-3,masVnrType-4,masVnrType-5,exterQual-0,exterQual-1,exterQual-2,exterQual-3,exterQual-4,exterCond-0,exterCond-1,exterCond-2,exterCond-3,exterCond-4,exterCond-5,foundation-0,foundation-1,foundation-2,foundation-3,foundation-4,foundation-5,foundation-6,bsmtQual-0,bsmtQual-1,bsmtQual-2,bsmtQual-3,bsmtQual-4,bsmtQual-5,bsmtCond-0,bsmtCond-1,bsmtCond-2,bsmtCond-3,bsmtCond-4,bsmtCond-5,bsmtExposure-0,bsmtExposure-1,bsmtExposure-2,bsmtExposure-3,bsmtExposure-4,bsmtExposure-5,bsmtFinType1-0,bsmtFinType1-1,bsmtFinType1-2,bsmtFinType1-3,bsmtFinType1-4,bsmtFinType1-5,bsmtFinType1-6,bsmtFinType1-7,bsmtFinType2-0,bsmtFinType2-1,bsmtFinType2-2,bsmtFinType2-3,bsmtFinType2-4,bsmtFinType2-5,bsmtFinType2-6,bsmtFinType2-7,heating-0,heating-1,heating-2,heating-3,heating-4,heating-5,heating-6,heatingQC-0,heatingQC-1,heatingQC-2,heatingQC-3,heatingQC-4,heatingQC-5,centralAir-0,centralAir-1,centralAir-2,electrical-0,electrical-1,electrical-2,electrical-3,electrical-4,electrical-5,electrical-6,kitchenQual-0,kitchenQual-1,kitchenQual-2,kitchenQual-3,kitchenQual-4,functional-0,functional-1,functional-2,functional-3,functional-4,functional-5,functional-6,functional-7,fireplaceQu-0,fireplaceQu-1,fireplaceQu-2,fireplaceQu-3,fireplaceQu-4,fireplaceQu-5,fireplaceQu-6,garageType-0,garageType-1,garageType-2,garageType-3,garageType-4,garageType-5,garageType-6,garageType-7,garageFinish-0,garageFinish-1,garageFinish-2,garageFinish-3,garageFinish-4,garageQual-0,garageQual-1,garageQual-2,garageQual-3,garageQual-4,garageQual-5,garageQual-6,garageCond-0,garageCond-1,garageCond-2,garageCond-3,garageCond-4,garageCond-5,garageCond-6,pavedDrive-0,pavedDrive-1,pavedDrive-2,pavedDrive-3,poolQC-0,poolQC-1,poolQC-2,poolQC-3,poolQC-4,fence-0,fence-1,fence-2,fence-3,fence-4,fence-5,miscFeature-0,miscFeature-1,miscFeature-2,miscFeature-3,miscFeature-4,miscFeature-5,saleType-0,saleType-1,saleType-2,saleType-3,saleType-4,saleType-5,saleType-6,saleType-7,saleCondition-0,saleCondition-1,saleCondition-2,saleCondition-3,saleCondition-4,saleCondition-5,saleCondition-6
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,0,150,856,856,854,0,1710,1,0,2,1,3,1,8,0,2003.0,2,548,0,61,0,0,0,0,0,2,2008,208500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [0]:

print(str(type(dfb.at[0,'Id'])) == "<class 'numpy.int64'>")
print(f"|{type(dfb.at[0,'Id'])}|")

True
|<class 'numpy.int64'>|


In [10]:
def test(d,c):
#   print(c)
#   print(str(type(d)))
  dtype = str(type(d))
  if   dtype != "<class 'numpy.int64'>" and
       dtype != "<class 'numpy.float64'>" and
       dtype != "<class 'int'>" and
       dtype != "<class 'float'>":
    print('unknown',c,str(type(d)))
    return None
  return d

for c in dfb.columns:
  try:
    if cats.index(c) >= 0:
       continue
  except:
      pass
  finally:
#     dfb[c] = dfb[c].apply(lambda d: None if (str(type(d)) != "<class 'numpy.int64'>") and 
#                           (str(type(d)) != "<class 'numpy.float64'>") else d)
    dfb[c] = dfb[c].apply(lambda d: test(d,c))
    nnn = dfb[dfb[c].notnull()][c]
    print('Not Null', c, nnn.shape)
    nnnn= dfb[dfb[c].isnull()][c]
    print('Null', c, nnnn.shape)
    

SyntaxError: ignored

In [0]:
#df.isnull().count()

In [0]:
price = np.log(df['SalePrice'])
dfxp = dfb.drop(['Id', 'SalePrice'],inplace=False, axis=1)
X_train, X_test, Y_train, Y_test = train_test_split(dfxp, price, test_size=.5, random_state=42)
X_train.shape

(730, 380)

In [0]:
model = LinearRegression()

In [0]:
X = X_train
y = Y_train

# Our data needs to have this shape to be accepted by sklearn
print(X.shape)
print(y.shape)
model.fit(X, y)
print("Slope Coefficient: ", model.coef_)
print("\nIntercept Value: ", model.intercept_)

(730, 380)
(730,)


ValueError: ignored

In [0]:
X = X_train
y = Y_train

# Our data needs to have this shape to be accepted by sklearn
print(X.shape)
print(y.shape)
model.fit(X, y)
print("Slope Coefficient: ", model.coef_)
print("\nIntercept Value: ", model.intercept_)

(730, 380)
(730,)


ValueError: ignored

In [0]:
X = X_train
y = Y_train

# Our data needs to have this shape to be accepted by sklearn
print(X.shape)
print(y.shape)
model.fit(X, y)
print("Slope Coefficient: ", model.coef_)
print("\nIntercept Value: ", model.intercept_)

(730, 380)
(730,)


ValueError: ignored

# Stretch Goals

- Write a blog post explaining one of today's topics.
- Find a new regression dataset from the UCI machine learning repository and use it to test out your new modeling skillz.
 [ - UCI Machine Learning Repository - Regression Datasets](https://)
- Make a list for yourself of common feature engineering techniques. Browse Kaggle kernels to learn more methods.
- Start studying for tomorrow's topic: Gradient Descent
- Try and make the ultimate model with this dataset. clean as many features as possible, engineer the most sensible features as possible and see how accurate of a prediction you can make. 
- Learn about the "Dummy Variable Trap" and how it applies to linear regression modeling.
- Learning about using linear regression to model time series data